## Prerequisites
Before starting, let's update the botocore and boto3 packages to ensure we have the latest version

In [1]:
# !python3 -m pip install --upgrade pip
!python3 -m pip install --upgrade -q botocore
!python3 -m pip install --upgrade -q boto3
!python3 -m pip install --upgrade -q awscli
!python3 -m pip install python-dotenv

Let's now check the boto3 version to ensure the correct version has been installed. Your version should be greater than or equal to 1.34.90.

In [2]:
import boto3
import json
import time
import zipfile
from io import BytesIO
import uuid
import pprint
import logging
from dotenv import load_dotenv
import os
print(boto3.__version__)


load_dotenv()  # take environment variables from .env.

1.35.5


True

In [3]:
aws_access_key_id = os.getenv('AWS_ACCESS_KEY_ID')
aws_secret_access_key = os.getenv('AWS_SECRET_ACCESS_KEY')
aws_region = os.getenv('AWS_REGION')

In [4]:
# setting logger
logging.basicConfig(format='[%(asctime)s] p%(process)s {%(filename)s:%(lineno)d} %(levelname)s - %(message)s', level=logging.INFO)
logger = logging.getLogger(__name__)

Let's now create the boto3 clients for the required AWS services

In [5]:
# getting boto3 clients for required AWS services
sts_client = boto3.client(
    "sts",
    aws_access_key_id=aws_access_key_id,
    aws_secret_access_key=aws_secret_access_key,
    region_name=aws_region,
)
iam_client = boto3.client(
    "iam",
    aws_access_key_id=aws_access_key_id,
    aws_secret_access_key=aws_secret_access_key,
    region_name=aws_region,
)
lambda_client = boto3.client(
    "lambda",
    aws_access_key_id=aws_access_key_id,
    aws_secret_access_key=aws_secret_access_key,
    region_name=aws_region,
)
bedrock_agent_client = boto3.client(
    "bedrock-agent",
    aws_access_key_id=aws_access_key_id,
    aws_secret_access_key=aws_secret_access_key,
    region_name=aws_region,
)
bedrock_agent_runtime_client = boto3.client(
    "bedrock-agent-runtime",
    aws_access_key_id=aws_access_key_id,
    aws_secret_access_key=aws_secret_access_key,
    region_name=aws_region,
)

Next we can set some configuration variables for the agent and for the lambda function being created

In [6]:
session = boto3.session.Session(
    aws_access_key_id=aws_access_key_id,
    aws_secret_access_key=aws_secret_access_key,
    region_name=aws_region,
)
region = session.region_name
account_id = sts_client.get_caller_identity()["Account"]
region, account_id

('us-east-1', '738012852934')

In [7]:
# configuration variables

agent_name = "monkeypox-impact-monitor"
agent_description = "Agent for monitoring travel restrictions, analyzing public sentiment, and validating news sources to support travel insurance decisions related to monkeypox outbreaks."
agent_instruction = "You are an intelligent agent designed to assist a travel insurance agency by monitoring global travel restrictions, analyzing public sentiment, and validating news related to monkeypox outbreaks. Your insights help the agency adjust coverage, refine marketing strategies, and ensure credible decision-making to protect and inform customers."
agent_action_group_name = "MonkeypoxImpactActionGroup"
agent_action_group_description = "Actions for monitoring travel restrictions, performing sentiment analysis on tourism destinations, and validating the credibility of news sources to support travel insurance adjustments."


suffix = f"{region}-{account_id}"
agent_bedrock_allow_policy_name = f"{agent_name}-ba-{suffix}"
agent_role_name = f'AmazonBedrockExecutionRoleForAgents_{agent_name}'
agent_foundation_model = "anthropic.claude-3-sonnet-20240229-v1:0"
agent_alias_name = f"{agent_name}-alias"
lambda_function_role = f'{agent_name}-lambda-role-{suffix}'
lambda_function_name = f'{agent_name}-{suffix}'

## Creating Lambda Function

We will now create a lambda function. To do so we will:
1. Create the `lambda_function.py` file which contains the logic for access to other sources
2. Create the IAM role for our Lambda function
3. Create the lambda function infrastructure with the required permissions

Let's now create our lambda function. It implements the functionality for `get_available_vacations_days` for a given employee_id and `book_vacations` for an employee giving a start and end date

In [9]:
# %%writefile lambda_function.py


# def lambda_handler(event, context):
# #    Implement code

Next let's create the lambda IAM role and policy to invoke a Bedrock model

In [10]:
# Create IAM Role for the Lambda function
try:
    assume_role_policy_document = {
        "Version": "2012-10-17",
        "Statement": [
            {
                "Effect": "Allow",
                "Action": "bedrock:InvokeModel",
                "Principal": {
                    "Service": "lambda.amazonaws.com"
                },
                "Action": "sts:AssumeRole"
            }
        ]
    }

    assume_role_policy_document_json = json.dumps(assume_role_policy_document)

    lambda_iam_role = iam_client.create_role(
        RoleName=lambda_function_role,
        AssumeRolePolicyDocument=assume_role_policy_document_json
    )

    # Pause to make sure role is created
    time.sleep(10)
except:
    lambda_iam_role = iam_client.get_role(RoleName=lambda_function_role)

iam_client.attach_role_policy(
    RoleName=lambda_function_role,
    PolicyArn='arn:aws:iam::aws:policy/service-role/AWSLambdaBasicExecutionRole'
)

{'ResponseMetadata': {'RequestId': '57f53524-cf9e-45db-8da8-a065c37ac6f6',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'date': 'Sun, 25 Aug 2024 22:59:02 GMT',
   'x-amzn-requestid': '57f53524-cf9e-45db-8da8-a065c37ac6f6',
   'content-type': 'text/xml',
   'content-length': '212'},
  'RetryAttempts': 0}}

We can now package the lambda function to a Zip file and create the lambda function using boto3

In [11]:
# Package up the lambda function code
s = BytesIO()
z = zipfile.ZipFile(s, 'w')

z.write("lambda_function.py")

z.write("bing.py")
z.write("data_lake.py")
z.write("helpers.py")
z.write("pinecone.py")

z.close()
zip_content = s.getvalue()

# Create Lambda Function
lambda_function = lambda_client.create_function(
    FunctionName=lambda_function_name,
    Runtime='python3.9',
    Timeout=180,
    Role=lambda_iam_role['Role']['Arn'],
    Code={'ZipFile': zip_content},
    Handler='lambda_function.lambda_handler'
)

In [52]:
# UPDATE LAMBDA!

# Package up the lambda function code
s = BytesIO()
z = zipfile.ZipFile(s, 'w')

# Add your updated code files to the zip
z.write("./lambda/lambda_function.py")
z.write("./lambda/bing.py")
z.write("./lambda/data_lake.py")
z.write("./lambda/helpers.py")
z.write("./lambda/pinecone.py")
z.write("./lambda/.env")

z.close()
zip_content = s.getvalue()

# Update Lambda Function Code
response = lambda_client.update_function_code(
    FunctionName=lambda_function_name,
    ZipFile=zip_content
)

# Print response to confirm update
print("Update Response:", response)

Update Response: {'ResponseMetadata': {'RequestId': '47efb611-222d-4281-abe9-184b3e1cb720', 'HTTPStatusCode': 200, 'HTTPHeaders': {'date': 'Mon, 26 Aug 2024 01:26:30 GMT', 'content-type': 'application/json', 'content-length': '1480', 'connection': 'keep-alive', 'x-amzn-requestid': '47efb611-222d-4281-abe9-184b3e1cb720'}, 'RetryAttempts': 0}, 'FunctionName': 'monkeypox-impact-monitor-us-east-1-738012852934', 'FunctionArn': 'arn:aws:lambda:us-east-1:738012852934:function:monkeypox-impact-monitor-us-east-1-738012852934', 'Runtime': 'python3.9', 'Role': 'arn:aws:iam::738012852934:role/monkeypox-impact-monitor-lambda-role-us-east-1-738012852934', 'Handler': 'lambda_function.lambda_handler', 'CodeSize': 5290, 'Description': '', 'Timeout': 180, 'MemorySize': 128, 'LastModified': '2024-08-26T01:26:30.000+0000', 'CodeSha256': 'ANeux/Ks2vjWtn4VT3EBerHMG4nmexudjnHV5POuIxU=', 'Version': '$LATEST', 'TracingConfig': {'Mode': 'PassThrough'}, 'RevisionId': 'c4241823-60c0-4026-bd68-29cafe9337ac', 'Stat

## Create Agent
We will now create the agent. To do so, we first need to create the agent policies that allow bedrock model invocation for a specific foundation model and the agent IAM role with the policy associated to it. 

In [12]:
# Create IAM policies for agent
bedrock_agent_bedrock_allow_policy_statement = {
    "Version": "2012-10-17",
    "Statement": [
        {
            "Sid": "AmazonBedrockAgentBedrockFoundationModelPolicy",
            "Effect": "Allow",
            "Action": "bedrock:InvokeModel",
            "Resource": [
                f"arn:aws:bedrock:{region}::foundation-model/{agent_foundation_model}"
            ]
        }
    ]
}

bedrock_policy_json = json.dumps(bedrock_agent_bedrock_allow_policy_statement)

agent_bedrock_policy = iam_client.create_policy(
    PolicyName=agent_bedrock_allow_policy_name,
    PolicyDocument=bedrock_policy_json
)



In [13]:
# Create IAM Role for the agent and attach IAM policies
assume_role_policy_document = {
    "Version": "2012-10-17",
    "Statement": [{
          "Effect": "Allow",
          "Principal": {
            "Service": "bedrock.amazonaws.com"
          },
          "Action": "sts:AssumeRole"
    }]
}

assume_role_policy_document_json = json.dumps(assume_role_policy_document)
agent_role = iam_client.create_role(
    RoleName=agent_role_name,
    AssumeRolePolicyDocument=assume_role_policy_document_json
)

# Pause to make sure role is created
time.sleep(10)
    
iam_client.attach_role_policy(
    RoleName=agent_role_name,
    PolicyArn=agent_bedrock_policy['Policy']['Arn']
)

{'ResponseMetadata': {'RequestId': 'f5f2de62-2417-4408-8ac4-b0a27107d1e6',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'date': 'Sun, 25 Aug 2024 23:00:42 GMT',
   'x-amzn-requestid': 'f5f2de62-2417-4408-8ac4-b0a27107d1e6',
   'content-type': 'text/xml',
   'content-length': '212'},
  'RetryAttempts': 0}}

### Creating the agent
Once the needed IAM role is created, we can use the Bedrock Agent client to create a new agent. To do so we use the `create_agent` function. It requires an agent name, underlying foundation model and instructions. You can also provide an agent description. Note that the agent created is not yet prepared. Later, we will prepare and use the agent.

In [14]:
response = bedrock_agent_client.create_agent(
    agentName=agent_name,
    agentResourceRoleArn=agent_role['Role']['Arn'],
    description=agent_description,
    idleSessionTTLInSeconds=1800,
    foundationModel=agent_foundation_model,
    instruction=agent_instruction,
)
response

{'ResponseMetadata': {'RequestId': '67b8166d-46c4-4731-a06e-97d33d702821',
  'HTTPStatusCode': 202,
  'HTTPHeaders': {'date': 'Sun, 25 Aug 2024 23:01:08 GMT',
   'content-type': 'application/json',
   'content-length': '1006',
   'connection': 'keep-alive',
   'x-amzn-requestid': '67b8166d-46c4-4731-a06e-97d33d702821',
   'x-amz-apigw-id': 'dFmkMGraIAMEOoQ=',
   'x-amzn-trace-id': 'Root=1-66cbb7b4-2865a5d7618ce6e95dc16007'},
  'RetryAttempts': 0},
 'agent': {'agentArn': 'arn:aws:bedrock:us-east-1:738012852934:agent/VNSFCLRGBB',
  'agentId': 'VNSFCLRGBB',
  'agentName': 'monkeypox-impact-monitor',
  'agentResourceRoleArn': 'arn:aws:iam::738012852934:role/AmazonBedrockExecutionRoleForAgents_monkeypox-impact-monitor',
  'agentStatus': 'CREATING',
  'createdAt': datetime.datetime(2024, 8, 25, 23, 1, 8, 344339, tzinfo=tzutc()),
  'description': 'Agent for monitoring travel restrictions, analyzing public sentiment, and validating news sources to support travel insurance decisions related to 

Let's now store the agent id in a local variable to use it on subsequent steps.

In [15]:
agent_id = response['agent']['agentId']
agent_id

'VNSFCLRGBB'

## Create Agent Action Group
We will now create an agent action group that uses the lambda function created earlier. The [`create_agent_action_group`](https://boto3.amazonaws.com/v1/documentation/api/latest/reference/services/bedrock-agent/client/create_agent_action_group.html) function provides this functionality. We will use `DRAFT` as the agent version since we haven't yet created an agent version or alias. To inform the agent about the action group capabilities, we provide an action group description.

In this example, we provide the Action Group functionality using a `functionSchema`. You can alternatively provide an `APISchema`.

To define the functions using a function schema, you need to provide the `name`, `description` and `parameters` for each function.

In [16]:
agent_functions = [
    {
        'name': 'monitor_travel_restrictions',
        'description': 'Monitors global travel restrictions related to monkeypox outbreaks and adjusts insurance coverage options.',
        'parameters': {
            "region": {
                "description": "The specific region or country to monitor for travel restrictions.",
                "required": True,
                "type": "string"
            },
            "alert_threshold": {
                "description": "The number of news articles or alerts required to trigger an insurance policy adjustment.",
                "required": True,
                "type": "integer"
            },
            "coverage_type": {
                "description": "The type of insurance coverage to adjust (e.g., medical, cancellation).",
                "required": False,
                "type": "string"
            }
        }
    },
    {
        'name': 'analyze_public_sentiment',
        'description': 'Performs sentiment analysis on news articles to assess public perception of tourism destinations in light of monkeypox outbreaks.',
        'parameters': {
            "destination": {
                "description": "The tourist destination or region to analyze for sentiment.",
                "required": True,
                "type": "string"
            },
            "time_frame": {
                "description": "The period over which to analyze sentiment (e.g., last week, last month).",
                "required": False,
                "type": "string"
            }
        }
    },
    {
        'name': 'validate_news_sources',
        'description': 'Verifies the credibility and accuracy of news articles related to monkeypox outbreaks using multiple sources.',
        'parameters': {
            "news_id": {
                "description": "The unique identifier of the news article to validate.",
                "required": True,
                "type": "string"
            },
            "source_criteria": {
                "description": "Criteria for source validation (e.g., governmental, medical).",
                "required": False,
                "type": "string"
            }
        }
    }
]

In [17]:
# Pause to make sure agent is created
time.sleep(30)
# Now, we can configure and create an action group here:
agent_action_group_response = bedrock_agent_client.create_agent_action_group(
    agentId=agent_id,
    agentVersion='DRAFT',
    actionGroupExecutor={
        'lambda': lambda_function['FunctionArn']
    },
    actionGroupName=agent_action_group_name,
    functionSchema={
        'functions': agent_functions
    },
    description=agent_action_group_description
)

In [18]:
agent_action_group_response

{'ResponseMetadata': {'RequestId': '20ef3808-a0f2-466f-8115-d7aaa6d74fe7',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'date': 'Sun, 25 Aug 2024 23:03:53 GMT',
   'content-type': 'application/json',
   'content-length': '2208',
   'connection': 'keep-alive',
   'x-amzn-requestid': '20ef3808-a0f2-466f-8115-d7aaa6d74fe7',
   'x-amz-apigw-id': 'dFm98FFCIAMEnLw=',
   'x-amzn-trace-id': 'Root=1-66cbb858-7c87d1ad70cf6b8c760eaf66'},
  'RetryAttempts': 0},
 'agentActionGroup': {'actionGroupExecutor': {'lambda': 'arn:aws:lambda:us-east-1:738012852934:function:monkeypox-impact-monitor-us-east-1-738012852934'},
  'actionGroupId': 'PTQPY8PTBD',
  'actionGroupName': 'MonkeypoxImpactActionGroup',
  'actionGroupState': 'ENABLED',
  'agentId': 'VNSFCLRGBB',
  'agentVersion': 'DRAFT',
  'createdAt': datetime.datetime(2024, 8, 25, 23, 3, 53, 124364, tzinfo=tzutc()),
  'description': 'Actions for monitoring travel restrictions, performing sentiment analysis on tourism destinations, and validating the cred

## Allowing Agent to invoke Action Group Lambda
Before using the action group, we need to allow the agent to invoke the lambda function associated with the action group. This is done via resource-based policy. Let's add the resource-based policy to the lambda function created

In [13]:
# Create allow invoke permission on lambda
lambda_function_name="factored-hackaton-2024-unlock-bedrock-lambda"
response = lambda_client.add_permission(
    FunctionName=lambda_function_name,
    StatementId='allow_bedrock',
    Action='lambda:InvokeFunction',
    Principal='bedrock.amazonaws.com',
    SourceArn=f"arn:aws:bedrock:{region}:{account_id}:agent/{agent_id}",
)


In [14]:
response

{'ResponseMetadata': {'RequestId': '1f4fa8cd-07cd-4151-a3f2-bc1dd4a00127',
  'HTTPStatusCode': 201,
  'HTTPHeaders': {'date': 'Mon, 26 Aug 2024 02:10:54 GMT',
   'content-type': 'application/json',
   'content-length': '372',
   'connection': 'keep-alive',
   'x-amzn-requestid': '1f4fa8cd-07cd-4151-a3f2-bc1dd4a00127'},
  'RetryAttempts': 0},
 'Statement': '{"Sid":"allow_bedrock","Effect":"Allow","Principal":{"Service":"bedrock.amazonaws.com"},"Action":"lambda:InvokeFunction","Resource":"arn:aws:lambda:us-east-1:738012852934:function:factored-hackaton-2024-unlock-bedrock-lambda","Condition":{"ArnLike":{"AWS:SourceArn":"arn:aws:bedrock:us-east-1:738012852934:agent/VNSFCLRGBB"}}}'}

## Preparing Agent

Let's create a DRAFT version of the agent that can be used for internal testing.


In [15]:
agent_id = "VNSFCLRGBB"
response = bedrock_agent_client.prepare_agent(
    agentId=agent_id
)
print(response)

{'ResponseMetadata': {'RequestId': 'cd223d24-bf00-4b6f-90e9-d3ecd498d6f1', 'HTTPStatusCode': 202, 'HTTPHeaders': {'date': 'Mon, 26 Aug 2024 02:11:07 GMT', 'content-type': 'application/json', 'content-length': '119', 'connection': 'keep-alive', 'x-amzn-requestid': 'cd223d24-bf00-4b6f-90e9-d3ecd498d6f1', 'x-amz-apigw-id': 'dGCZYHVRIAMECKw=', 'x-amzn-trace-id': 'Root=1-66cbe43b-0fc5dfd01f9c4e634c36713c'}, 'RetryAttempts': 0}, 'agentId': 'VNSFCLRGBB', 'agentStatus': 'PREPARING', 'agentVersion': 'DRAFT', 'preparedAt': datetime.datetime(2024, 8, 26, 2, 11, 7, 864339, tzinfo=tzutc())}


In [16]:
# Pause to make sure agent is prepared
time.sleep(30)

# Extract the agentAliasId from the response
agent_alias_id = "TSTALIASID"

## Invoke Agent

Now that we've created the agent, let's use the `bedrock-agent-runtime` client to invoke this agent and perform some tasks.

In [17]:
def invokeAgent(inputText: str):

    session_id:str = str(uuid.uuid1())
    enable_trace:bool = False
    end_session:bool = False

    # invoke the agent API
    agentResponse = bedrock_agent_runtime_client.invoke_agent(
        inputText=inputText,
        agentId=agent_id,
        agentAliasId=agent_alias_id, 
        sessionId=session_id,
        enableTrace=enable_trace, 
        endSession= end_session
    )

    logger.info(pprint.pprint(agentResponse))


    # %%time
    event_stream = agentResponse['completion']
    try:
        for event in event_stream:        
            if 'chunk' in event:
                data = event['chunk']['bytes']
                logger.info(f"Final answer ->\n{data.decode('utf8')}")
                agent_answer = data.decode('utf8')
                end_event_received = True
                # End event indicates that the request finished successfully
            elif 'trace' in event:
                logger.info(json.dumps(event['trace'], indent=2))
            else:
                raise Exception("unexpected event.", event)
    except Exception as e:
        raise Exception("unexpected event.", e)


    # And here is the response if you just want to see agent's reply
    return agent_answer

In [23]:
#Test the agent,

invokeAgent("I want to monitor travel restrictions in Nigeria for monkey pox?")

[2024-08-25 21:54:18,679] p53017 {954796933.py:17} INFO - None


{'ResponseMetadata': {'HTTPHeaders': {'connection': 'keep-alive',
                                      'content-type': 'application/json',
                                      'date': 'Mon, 26 Aug 2024 02:54:18 GMT',
                                      'transfer-encoding': 'chunked',
                                      'x-amz-bedrock-agent-session-id': '7cb8bdc6-6356-11ef-92d6-eaa40401cf87',
                                      'x-amzn-bedrock-agent-content-type': 'application/json',
                                      'x-amzn-requestid': '4854d6b4-96bc-49c4-a3ec-6bbebbd5cd33'},
                      'HTTPStatusCode': 200,
                      'RequestId': '4854d6b4-96bc-49c4-a3ec-6bbebbd5cd33',
                      'RetryAttempts': 0},
 'completion': <botocore.eventstream.EventStream object at 0x1217d0280>,
 'contentType': 'application/json',
 'sessionId': '7cb8bdc6-6356-11ef-92d6-eaa40401cf87'}


Exception: ('unexpected event.', EventStreamError("An error occurred (dependencyFailedException) when calling the InvokeAgent operation: Your request couldn't be completed. Lambda function arn:aws:lambda:us-east-1:738012852934:function:factored-hackaton-2024-unlock-bedrock-lambda encountered a problem while processing request.The error message from the Lambda function is Unhandled. Check the Lambda function log for error details, then try your request again after fixing the error."))